In [8]:
from sqlalchemy import create_engine
import pandas as pd

### Step 1: Establish a Connection


#### Set the conection details

In [10]:
# Database connection details
host = 'localhost'
user = 'danicoco'
password = 'password'
database = 'sakila'
port = '3306'  

# Connect to the Database Legos

In [13]:
# Connect to the database
DATABASE_URI = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
# Create the database engine
engine = create_engine(DATABASE_URI)

In [14]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, customer_id, return_date, staff_id, inventory_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    with engine.connect() as conn:
        df = pd.read_sql(query, conn)
    return df

def rental_count_month(df, month, year):
    rental_counts = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_counts

def compare_rentals(df1, df2, month1, year1, month2, year2):
    df_merged = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    df_merged['difference'] = df_merged[f'rentals_{month2:02d}_{year2}'] - df_merged[f'rentals_{month1:02d}_{year1}']
    return df_merged


In [15]:
# Retrieve rental data for May and June
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)


In [16]:
# Get rental counts for May and June
rental_counts_may = rental_count_month(df_may, 5, 2005)
rental_counts_june = rental_count_month(df_june, 6, 2005)

In [17]:
# Compare rental counts between May and June
comparison_df = compare_rentals(rental_counts_may, rental_counts_june, 5, 2005, 6, 2005)


In [18]:
# Display the comparison DataFrame
print(comparison_df)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0         5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0         2.0
3              5              3.0              5.0         2.0
4              6              3.0              4.0         1.0
..           ...              ...              ...         ...
593          583              0.0              6.0         6.0
594          585              0.0              4.0         4.0
595          591              0.0              3.0         3.0
596          592              0.0              5.0         5.0
597          598              0.0              1.0         1.0

[598 rows x 4 columns]
